In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time  # for adding delay between requests

base_url = "https://www.bbcgoodfood.com/recipes/category/cuisine-collections?page=1"

# Function to check if a link is a cuisine collection link
def is_cuisine_link(link):
    # Check if the link URL contains '/recipes/collection/'
    return '/recipes/collection/' in link['href']

# Function to scrape cuisine collection URLs from the main page
def scrape_cuisine_collections(url):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        cuisine_collections = []
        cuisine_elements = soup.find_all('a', class_='link d-block')  # Adjusted class name
        
        for cuisine in cuisine_elements:
            if is_cuisine_link(cuisine):
                cuisine_title = cuisine.text.strip()
                cuisine_url = f"https://www.bbcgoodfood.com{cuisine['href']}"
                
                cuisine_collections.append({
                    'Title': cuisine_title,
                    'URL': cuisine_url
                })
        
        return cuisine_collections
    else:
        print(f"Failed to retrieve page: {response.status_code}")
        return []

# Function to scrape recipe data from a single cuisine collection page
def scrape_recipes_from_cuisine(url):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        recipes = []
        recipe_elements = soup.find_all('a', class_='link d-block')  # Adjusted class name
        
        for recipe in recipe_elements:
            recipe_title = recipe.text.strip()
            recipe_url = f"https://www.bbcgoodfood.com{recipe['href']}"
            
            recipes.append({
                'Title': recipe_title,
                'URL': recipe_url
            })
        
        return recipes
    else:
        print(f"Failed to retrieve page: {response.status_code}")
        return []

# Function to write recipes to CSV
def write_to_csv(recipes, filename):
    fields = ['Title', 'URL']
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        writer.writeheader()
        writer.writerows(recipes)
    print(f"Saved {len(recipes)} recipes to {filename}")

# Main function to scrape multiple pages and write to CSV
def main():
    all_cuisine_collections = []
    for page_num in range(1, 2):  # scraping pages 1 to 5 for example
        url = f"https://www.bbcgoodfood.com/recipes/category/cuisine-collections?page={page_num}"
        print(f"Scraping cuisine collections from {url}")
        cuisine_collections = scrape_cuisine_collections(url)
        all_cuisine_collections.extend(cuisine_collections)
    
    all_recipes = []
    for cuisine in all_cuisine_collections:
        cuisine_url = cuisine['URL']
        print(f"Scraping recipes from cuisine collection: {cuisine['Title']}")
        recipes = scrape_recipes_from_cuisine(cuisine_url)
        all_recipes.extend(recipes)
        time.sleep(1)  # Adding a delay of 1 second between requests to be polite to the server
    
    if all_recipes:
        write_to_csv(all_recipes, 'GOODFOOD.csv')
    else:
        print("No recipes scraped. Check your scraping logic.")

if __name__ == "__main__":
    main()


Scraping cuisine collections from https://www.bbcgoodfood.com/recipes/category/cuisine-collections?page=1
Scraping recipes from cuisine collection: American
Scraping recipes from cuisine collection: British
Scraping recipes from cuisine collection: Caribbean
Scraping recipes from cuisine collection: Jerk chicken
Scraping recipes from cuisine collection: Chinese
Scraping recipes from cuisine collection: French
Scraping recipes from cuisine collection: Greek
Scraping recipes from cuisine collection: Japanese
Scraping recipes from cuisine collection: Katsu curry
Scraping recipes from cuisine collection: Scandinavian recipes
Scraping recipes from cuisine collection: German recipes
Scraping recipes from cuisine collection: Mexican
Scraping recipes from cuisine collection: Moroccan
Scraping recipes from cuisine collection: Spanish
Scraping recipes from cuisine collection: Thai
Scraping recipes from cuisine collection: Mediterranean
Scraping recipes from cuisine collection: Chicken tagine rec